In [7]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt

from micro_price_trading import Preprocess, TwoAssetSimulation
from micro_price_trading.config import TWENTY_SECOND_DAY, DATA_PATH

In [8]:
raw = Preprocess('Test_TBT_TBF_data.csv')
data = raw.process()

In [9]:
with open('simulations.npy', 'rb') as file:
    input_data = np.load(file)

In [10]:
q_values = pd.read_csv(DATA_PATH.joinpath('q_values_9_27.csv'), index_col=0)

In [11]:
def CI(array, confidence = .95):
    half_width = 1.96*array.std()/np.sqrt(len(array))
    mu = array.mean()
    return mu - half_width, mu, mu + half_width

In [12]:
def optimal_twap(t):

    avg_prices_1, avg_prices_2 = [], []

    indices_to_buy_at = np.array([idx for idx in range(t, TWENTY_SECOND_DAY+1, t)])

    for sim in input_data:

        q_values_binary = (q_values.iloc[sim[indices_to_buy_at,0]]['Long/Short'] > 
                           q_values.iloc[sim[indices_to_buy_at,0]]['Short/Long']).values


        indices_to_buy_asset_1 = indices_to_buy_at[np.where(q_values_binary)[0]]
        indices_to_buy_asset_2 = indices_to_buy_at[np.where(~q_values_binary)[0]]

        avg_prices_1.append(sim[indices_to_buy_asset_1,1].mean())
        avg_prices_2.append(sim[indices_to_buy_asset_1,2].mean())

    return np.array(avg_prices_1), np.array(avg_prices_2)


In [13]:
avg_prices_1, avg_prices_2 = optimal_twap(5)
CI(avg_prices_1), CI(avg_prices_2)

((18.31291700450401, 18.315084017189346, 18.317251029874683),
 (16.660120735978804, 16.66130504218239, 16.66248934838598))

In [8]:
''' ATTEMPT AT VECTORIZING FOR EFFICIENY

def optimal_twap(t):
    indices_to_buy_at = np.array([idx for idx in range(t, TWENTY_SECOND_DAY+1, t)])

    q_values_binary = (q_values.iloc[input_data[:,indices_to_buy_at,0].flatten()]['Long/Short'] >
                       q_values.iloc[input_data[:,indices_to_buy_at,0].flatten()]['Short/Long']).values
    q_values_binary = q_values_binary.reshape(size)

    temp = np.where(q_values_binary)
    temp_inv = np.where(~q_values_binary)

    indices_to_buy_asset_1 = indices_to_buy_at[temp[1]]
    indices_to_buy_asset_2 = indices_to_buy_at[temp_inv[1]]

    avg_prices_1 = input_data[temp[0],
                              indices_to_buy_asset_1,
                              1]

    avg_prices_2 = input_data[temp_inv[1],
                              indices_to_buy_asset_2,
                              2]


    return avg_prices_1, avg_prices_2'''

" ATTEMPT AT VECTORIZING FOR EFFICIENY\n\ndef optimal_twap(t):\n    indices_to_buy_at = np.array([idx for idx in range(t, TWENTY_SECOND_DAY+1, t)])\n\n    q_values_binary = (q_values.iloc[input_data[:,indices_to_buy_at,0].flatten()]['Long/Short'] >\n                       q_values.iloc[input_data[:,indices_to_buy_at,0].flatten()]['Short/Long']).values\n    q_values_binary = q_values_binary.reshape(size)\n\n    temp = np.where(q_values_binary)\n    temp_inv = np.where(~q_values_binary)\n\n    indices_to_buy_asset_1 = indices_to_buy_at[temp[1]]\n    indices_to_buy_asset_2 = indices_to_buy_at[temp_inv[1]]\n\n    avg_prices_1 = input_data[temp[0],\n                              indices_to_buy_asset_1,\n                              1]\n\n    avg_prices_2 = input_data[temp_inv[1],\n                              indices_to_buy_asset_2,\n                              2]\n\n\n    return avg_prices_1, avg_prices_2"